# Unsupervised SRBM

We can train RBM unsupervised.

Originally, we use data and model distribution to maximize the loglikelihood.

\begin{align}
{\rm Max} \left\langle \ln ( q(\phi) ) \right\rangle_{p}
\end{align}

Instead we can use generated data to estimate p.

\begin{align}
\mathcal{L} &= {\rm Max} \left\langle \ln ( p(\phi) ) \right\rangle_{q} \\
&= {\rm Max} \left\langle -S_{\phi}(\phi) - \ln Z \right\rangle
\end{align}

\begin{align}
\frac{\partial \mathcal{L}}{\partial w_{ia}} &= -\frac{\partial \langle S_{\phi}(\phi^{\rm rbm}) \rangle_{q}}{\partial w_{ia}}
 - \langle \phi_i \phi_j w_{ja} \rangle_{q} \\
 &= \left\langle S_{\phi}(\phi) \frac{\partial S_{\rm RBM}}{\partial w_{ia}} \right\rangle - \langle \phi_i \phi_j w_{ja} \rangle \\
 &= \left\langle S_{\phi} (\phi) \phi_i \phi_j w_{ja} \right\rangle - \langle \phi_i \phi_j w_{ja} \rangle
\end{align}

or

\begin{align}
\frac{\partial \mathcal{L}}{\partial w_{ia}} &= \langle \phi_i \phi_j w_{ja} \rangle_{q}
 - \langle \phi_i \phi_j w_{ja} \rangle_{q} \\
 &= K^{-1}_{\phi, ij}w_{ja} - K^{-1}_{\rm RBM, ij} w_{ja}
\end{align}

We have to know the expression for one-point and two point function.

In [ ]:
import numpy as np
import scipy as sp
import os
import time
from skrmt.ensemble.spectral_law import MarchenkoPasturDistribution
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc, rcParams
from matplotlib.ticker import AutoMinorLocator
%matplotlib inline
import os, sys

import random

In [ ]:
scale = 1.
W = 6* scale
r = 0.75

if scale != 1.:
    fonts = 21
    ms = 10
else:
    fonts = 18
    ms = 7

rcParams.update({'figure.figsize': (W, W*r),   # 4:3 aspect ratio
                 'font.size' : fonts* scale,      # Set font size to 11pt
                 'axes.labelsize': fonts* scale,  # -> axis labels
                 'xtick.direction':'in',       # xticks
                 'xtick.top': True,
                 'xtick.major.size': 6 * scale,
                 'xtick.major.width': 1 * scale,
                 'xtick.minor.visible': True,
                 'xtick.minor.size': 3 * scale,
                 'xtick.minor.width': 1 * scale,
                 'xtick.major.pad': 5.5 * scale,
                 'ytick.direction':'in',       # yticks
                 'ytick.right': True,
                 'ytick.major.size': 6 * scale,
                 'ytick.major.width': 1 * scale,
                 'ytick.minor.visible': True,
                 'ytick.minor.size': 3 * scale,
                 'ytick.minor.width': 1 * scale,
                 'ytick.major.pad': 5 * scale,
                 'legend.fontsize': fonts* scale, # -> legends
                 'legend.frameon': False,
                 'lines.markersize':ms* scale,
                 'lines.markerfacecolor':'none',
                 'lines.linewidth':1.*scale,
                 'errorbar.capsize':3.*scale,
                 'font.family': 'lmodern',
                 'text.usetex': True,
                 'text.latex.preamble': (      # LaTeX preamble
                     r'\usepackage{lmodern}'
                     # ... more packages if needed
                 ),
                })

ls_ = ['-','-.','--',':']
a_ = [1,0.8,0.6,0.4]

rbm_c = '#0072B2'
rbm_c2 = '#D55E00'
rbm_c3 = '#009E73'

# Marchenko - Pasture distribution

In [ ]:
# lot of samples
project_name = 'SRBM-scale'
print(project_name)

plot_dir = "../images/" + project_name + "/"

os.system("mkdir -p "+plot_dir)

data_dir = "../data/"+project_name + "/"
model_dir_n = data_dir+"/bs/"
model_dir_lr = data_dir+"/lr/"

data_list_n = os.listdir(model_dir_n)
data_list_lr = os.listdir(model_dir_lr)

models_n = [x for x in data_list_n if os.path.isdir(os.path.join(model_dir_n, x))]
models_lr = [x for x in data_list_lr if os.path.isdir(os.path.join(model_dir_lr, x))]

ns = [int(x.split('bs')[1]) for x in models_n]
lrs = [float(x.split('lr')[1].split('bs')[0]) for x in models_lr]

idx_n = np.argsort(ns)
idx_lr = np.argsort(lrs)

ns = np.unique(np.array(ns)[idx_n])
lrs = np.unique(np.array(lrs)[idx_lr])

print(ns)
print(lrs)

models_n = np.array(models_n)[idx_n]
models_lr = np.array(models_lr)[idx_lr]
print(models_n)
print(models_lr)

In [ ]:
fixed_lr = 0.1
fixed_bs = 16
n_seed = 500 # scalar-fine
n_run = 15
epochs = 100000 # scalar -fine
save_int = int(epochs/5)
N = 10

In [ ]:
load_data = True
if load_data:
    full_data = np.load(data_dir+"data.npz")
    print("Data loaded from ",data_dir+"data.npz")

In [ ]:
mi = 3.
print(mi)
eps = 0.1
print(eps)

# Define target kernel

In [ ]:
m = 2
N=10
K_phi = np.zeros((N,N))

for i in range(N):
    for j in range(N):
        if i==j:
            K_phi[i][j] = 2 + m**2
        elif (i % N == (j+1) %N) or (i % N == (j-1) %N):
            K_phi[i][j] = -1
print(K_phi)

eig_phi = np.empty(N)
for i in range(N):
    eig_phi[i] = m**2 + 2 - 2*np.cos(2*np.pi*i/N)
    
eig_phi = np.sort(eig_phi)

In [ ]:
print(eig_phi)

In [ ]:
data_name_n = []
for n in models_n:
    data_name_n.append(model_dir_n + n + '/' + n + '.npz')
print(data_name_n)

In [ ]:
data_name_lr = []
for n in models_lr:
    data_name_lr.append(model_dir_lr + n + '/' + n + '.npz')
print(data_name_lr)

In [ ]:
print("loaded all:", len(data_name_n) == 75)

In [ ]:
eig_hist_n = np.empty((len(ns), n_seed*n_run, epochs//save_int + 1, N))

for i, d in enumerate(data_name_n):
    try:
        fi = np.load(d)
        idx1 = i//n_run
        idx2 = i%n_run
        print(idx1, idx2, d)
        eig_hist_n[idx1][idx2*n_seed: (idx2+1)*n_seed] = fi['s']
    except Exception as exp:
        print(exp)
        continue


In [ ]:
for i in range(len(eig_hist_n)):
    print(eig_hist_n[i].shape)
    
print(eig_hist_n.shape)

In [ ]:
eig_hist_lr = np.empty((len(lrs), n_seed*n_run, epochs//save_int + 1, N))
for i, d in enumerate(data_name_lr):
    try:
        fi = np.load(d)
        idx1 = i//n_run
        idx2 = i%n_run
        eig_hist_lr[idx1][idx2*n_seed: (idx2+1)*n_seed] = fi['s']
        print(idx1, idx2, d)
        y = mi**2 - fi['s'][:,-1,1:8:2]
        print(np.max(y))

    except Exception as exp:
        print(exp)
        continue

In [ ]:
for i in range(len(eig_hist_lr)):
    print(eig_hist_lr[i].shape)

# Freedman–Diaconis rule

Freedman, D., Diaconis, P. On the histogram as a density estimator:L 2 theory. Z. Wahrscheinlichkeitstheorie verw Gebiete 57, 453–476 (1981). https://doi.org/10.1007/BF01025868

\begin{align}
\text{Bin width} = 2 \frac{IQR(x)}{\sqrt[3]{n}} 
\end{align}

Where $IQR$ is interquartile range.

In [ ]:
def f_bin(y):
    n = len(y)
    Q1, Q3 = np.percentile(y, [25, 75])
    IQR = Q3 - Q1
    bw = 2.*IQR/np.cbrt(n)
    n_bin = int((max(y) - min(y))//bw)
    return n_bin, (min(y), max(y))

# Singular value distribution plot

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)

idx1 = 0
idx2 = 2
idx3 = 4

for i in eig_phi:
    ax1.axvline(i, ls='--', color='k')

y = mi**2 - eig_hist_lr[idx1,:,-1,0]
n_bins, xran = f_bin(y)
bw = np.abs(xran[1] - xran[0])/n_bins
y_whole = mi**2 - eig_hist_lr[idx1, :, -1, :].flatten()
_, xran = f_bin(y_whole)
n_bins = int((xran[1] - xran[0])//bw)
print(n_bins)

ax1.hist(y_whole, bins=n_bins, range=xran,
         density=True,
         color='steelblue',)

for i in eig_phi:
    ax2.axvline(i, ls='--', color='k')

y = mi**2 - eig_hist_lr[idx2,:,-1,0]
n_bins, xran = f_bin(y)
bw = np.abs(xran[1] - xran[0])/n_bins
y_whole = mi**2 - eig_hist_lr[idx2, :, -1, :].flatten()
_, xran = f_bin(y_whole)
n_bins = int((xran[1] - xran[0])//bw)
print(n_bins)

ax2.hist(y_whole, bins=n_bins, range=xran,
         density=True,
         color='steelblue',)

for i in eig_phi:
    ax3.axvline(i, ls='--', color='k')

y = mi**2 - eig_hist_lr[idx3,:,-1,0]
n_bins, xran = f_bin(y)
bw = np.abs(xran[1] - xran[0])/n_bins
y_whole = mi**2 - eig_hist_lr[idx3, :, -1, :].flatten()
_, xran = f_bin(y_whole)
n_bins = int((xran[1] - xran[0])//bw)
print(n_bins)

ax3.hist(y_whole, bins=n_bins, range=xran,
         density=True,
         color='steelblue',)

# xmin, xmax = (1.6, 2.4)
xmin, xmax = (3, 9)
xmean = 0.5*(xmax + xmin)
text_loc = 0.5*(xmax - xmean)
ytiks = np.arange(0,3,1)

ax1.set_xlim(xmin, xmax)
ax1.set_yticks(ytiks, ytiks)
ax1.set_ylim(0,2)

ax2.set_xlim(xmin, xmax)
ax2.set_yticks(ytiks, ytiks)
ax2.set_ylim(0,2)

ax3.set_xlim(xmin, xmax)
ax3.set_yticks(ytiks, ytiks)
ax3.set_ylim(0,2)

ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(AutoMinorLocator(2))
ax2.xaxis.set_minor_locator(AutoMinorLocator(2))
ax2.yaxis.set_minor_locator(AutoMinorLocator(2))
ax3.xaxis.set_minor_locator(AutoMinorLocator(2))
ax3.yaxis.set_minor_locator(AutoMinorLocator(2))

plt.xlabel(r'$\lambda = \mu^2 - x$')
plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.18)
plt.savefig(plot_dir+'lbd_comp_all3-density-r.pdf')

plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)

idx1 = 0
idx2 = 2
idx3 = 4

for i in eig_phi:
    ax1.axvline(mi**2 - i, ls='--', color='k')

y = eig_hist_lr[idx1,:,-1, 0]
n_bins, xran = f_bin(y)
bw = np.abs(xran[1] - xran[0])/n_bins
y_whole = eig_hist_lr[idx1, :, -1, :].flatten()
_, xran = f_bin(y_whole)
n_bins = int((xran[1] - xran[0])//bw)
print(n_bins)

ax1.hist(y_whole, bins=n_bins, range=xran,
         density=True,
         color='steelblue',)

for i in eig_phi:
    ax2.axvline(mi**2 - i, ls='--', color='k')

y = eig_hist_lr[idx2,:,-1,0]
n_bins, xran = f_bin(y)
bw = np.abs(xran[1] - xran[0])/n_bins
y_whole = eig_hist_lr[idx2, :, -1, :].flatten()
_, xran = f_bin(y_whole)
n_bins = int((xran[1] - xran[0])//bw)
print(n_bins)

ax2.hist(y_whole, bins=n_bins, range=xran,
         density=True,
         color='steelblue',)

for i in eig_phi:
    ax3.axvline(mi**2 - i, ls='--', color='k')

y = eig_hist_lr[idx3,:,-1,0]
n_bins, xran = f_bin(y)
bw = np.abs(xran[1] - xran[0])/n_bins
y_whole = eig_hist_lr[idx3, :, -1, :].flatten()
_, xran = f_bin(y_whole)
n_bins = int((xran[1] - xran[0])//bw)
print(n_bins)

ax3.hist(y_whole, bins=n_bins, range=xran,
         density=True,
         color='steelblue',)

xmin, xmax = (0, 6)
xmean = 0.5*(xmax + xmin)
text_loc = 0.5*(xmax - xmean)
ytiks = np.arange(0,3,1)

ax1.set_xlim(xmin, xmax)
ax1.set_yticks(ytiks, ytiks)
ax1.set_ylim(0,2)

ax2.set_xlim(xmin, xmax)
ax2.set_yticks(ytiks, ytiks)
ax2.set_ylim(0,2)

ax3.set_xlim(xmin, xmax)
ax3.set_yticks(ytiks, ytiks)
ax3.set_ylim(0,2)

ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(AutoMinorLocator(2))
ax2.xaxis.set_minor_locator(AutoMinorLocator(2))
ax2.yaxis.set_minor_locator(AutoMinorLocator(2))
ax3.xaxis.set_minor_locator(AutoMinorLocator(2))
ax3.yaxis.set_minor_locator(AutoMinorLocator(2))

plt.xlabel(r'$x$')
ax2.set_ylabel("distribution")
plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.18)
plt.savefig(plot_dir+'x_comp_all3-density-r.pdf')

plt.show()

# Wigner surmise

In [ ]:
finflag = (1,9,2)
num_split = 4

In [ ]:
# Wigner surmise

n_d = len(eig_hist_n)
dx_wig_n = np.empty((n_d, num_split))

rot = 1.4

for idx in range(finflag[0], finflag[1], finflag[2]):
    fig, axs = plt.subplots(1, 2, figsize=(W*rot, W/rot))
    for i in range(n_d):
        lbd_ens = eig_hist_n[i,:,-1, idx:idx+2]
        print(np.mean(lbd_ens))

        s_ens = np.abs(lbd_ens[:,0] - lbd_ens[:,1])

        y0 = s_ens
        n_bins0, xran0 = f_bin(y0)
        x0 = np.linspace(0, xran0[1], 100)

        hb0, pb0 = np.histogram(y0, range=xran0,
                              bins=n_bins0, density=True)

        px0 = np.empty_like(hb0)
        for j in range(len(px0)):
            px0[j] = 0.5*(pb0[j] + pb0[j+1])

        s_mean = np.sum(px0 * hb0)/np.sum(hb0)
        dx_wig_n[i][idx//2] = s_mean
        print('noise:', ns[i], 's:',s_mean)

        yp = y0 / s_mean

        n_bins, xran = f_bin(yp)
        x = np.linspace(0, 4, 100)

        def wigner(x):
            return 0.5*np.pi*x*np.exp(-np.pi * 0.25 * (x)**2)
        
        def wigner0(x):
            return 0.5*np.pi*(x/s_mean**2)*np.exp(-np.pi * 0.25 * (x/s_mean)**2)
        
        if i == n_d - 1:
            axs[0].plot(x, wigner(x), c='k', ls='-')
            axs[0].hist(yp, bins=n_bins, 
                        range=xran, density=True, color='steelblue')
            
            axs[1].plot(x0, wigner0(x0), c='k', ls='-', 
                        label=r'Wigner surmise')
            axs[1].hist(y0, bins=n_bins0, 
                        range=xran0, density=True, color='steelblue',
                       )
            
        else:
            axs[0].hist(yp, bins=n_bins, 
                        range=xran, density=True, 
                        color='steelblue', alpha=i/n_d)
            
            axs[1].plot(x0, wigner0(x0), 
                        c='k', ls='--', alpha=i/n_d)
            axs[1].hist(y0, bins=n_bins0, 
                        range=xran0, density=True, 
                        color='steelblue', alpha=i/n_d)

    xtik = np.arange(0,5,1)
    ytik = np.round(np.arange(0,2,0.3),1)
    axs[0].set_xlabel(r"$s = S / \langle S \rangle$")
    axs[0].set_ylabel(r"$P(s)$")
    axs[0].set_yticks(ytik, ytik)
    axs[0].set_xticks(xtik, xtik)
    axs[0].text(0.9,0.92,r"$\kappa=%.2f$"%(eig_phi[idx]),
                transform=axs[0].transAxes,
                horizontalalignment='right', verticalalignment='top')
    axs[0].set_xlim(0,4)
    axs[0].set_ylim(0,1.5)

    xtik = np.round(np.arange(0, 2., 0.3),2)
    ytik = np.arange(0, 12, 2)
    axs[1].set_xlabel(r"$S$")
    axs[1].set_ylabel(r"$P(S)$")
    axs[1].set_yticks(ytik, ytik)
    axs[1].set_xticks(xtik, xtik)
    axs[1].set_xlim(0,1.5)
    axs[1].set_ylim(0,10)

    plt.legend()
    
    axs[0].xaxis.set_minor_locator(AutoMinorLocator(2))
    axs[0].yaxis.set_minor_locator(AutoMinorLocator(2))
    axs[1].xaxis.set_minor_locator(AutoMinorLocator(2))
    axs[1].yaxis.set_minor_locator(AutoMinorLocator(2))

    plt.subplots_adjust(wspace=0.1, hspace=0)

    plt.tight_layout()

    plt.savefig(plot_dir+'wigner_n_sidebyside-%d.pdf'%(idx))
    plt.show()

In [ ]:
# Wigner surmise

n_d = len(eig_hist_lr)
dx_wig_lr = np.empty((n_d, num_split))

rot = 1.4

for idx in range(finflag[0], finflag[1], finflag[2]):
    fig, axs = plt.subplots(1, 2, figsize=(W*rot, W/rot))

    for i in range(n_d):
        lbd_ens = eig_hist_lr[i,:,-1, idx:idx+2]
        print(np.mean(lbd_ens))

        s_ens = np.abs(lbd_ens[:,0] - lbd_ens[:,1])

        y0 = s_ens
        n_bins0, xran0 = f_bin(y0)
        x0 = np.linspace(0, xran0[1], 100)

        hb0, pb0 = np.histogram(y0, range=xran0,
                              bins=n_bins0, density=True)

        px0 = np.empty_like(hb0)
        for j in range(len(px0)):
            px0[j] = 0.5*(pb0[j] + pb0[j+1])

        s_mean = np.sum(px0 * hb0)/np.sum(hb0)
        dx_wig_lr[i][idx//2] = s_mean
        print('noise:', lrs[i], 's:',s_mean)

        yp = y0 / s_mean

        n_bins, xran = f_bin(yp)
        x = np.linspace(0, 4, 100)

        def wigner(x):
            return 0.5*np.pi*x*np.exp(-np.pi * 0.25 * (x)**2)
        
        def wigner0(x):
            return 0.5*np.pi*(x/s_mean**2)*np.exp(-np.pi * 0.25 * (x/s_mean)**2)
        
        if i == 0:
            axs[0].plot(x, wigner(x), c='k', ls='-')
            axs[0].hist(yp, bins=n_bins, 
                        range=xran, density=True, color='steelblue')
            
            axs[1].plot(x0, wigner0(x0), c='k', ls='-', 
                        label=r'Wigner surmise')
            axs[1].hist(y0, bins=n_bins0, 
                        range=xran0, density=True, color='steelblue',
                       )
            
        else:
            axs[0].hist(yp, bins=n_bins, 
                        range=xran, density=True, 
                        color='steelblue', alpha=(n_d-i-1)/n_d)
            
            axs[1].plot(x0, wigner0(x0), 
                        c='k', ls='--', alpha=(n_d-i-1)/n_d)
            axs[1].hist(y0, bins=n_bins0, 
                        range=xran0, density=True, 
                        color='steelblue', alpha=(n_d-i-1)/n_d)

    xtik = np.arange(0,5,1)
    ytik = np.round(np.arange(0,2,0.3),1)
    axs[0].set_xlabel(r"$s = S / \langle S \rangle$")
    axs[0].set_ylabel(r"$P(s)$")
    axs[0].set_yticks(ytik, ytik)
    axs[0].set_xticks(xtik, xtik)
    axs[0].text(0.9,0.92,r"$\kappa=%.2f$"%(eig_phi[idx]),
                transform=axs[0].transAxes,
                horizontalalignment='right', verticalalignment='top')
    axs[0].set_xlim(0,4)
    axs[0].set_ylim(0,1.5)

    xtik = np.round(np.arange(0, 2., 0.3),2)
    ytik = np.arange(0, 12, 2)
    axs[1].set_xlabel(r"$S$")
    axs[1].set_ylabel(r"$P(S)$")
    axs[1].set_yticks(ytik, ytik)
    axs[1].set_xticks(xtik, xtik)
    axs[1].set_xlim(0,1.5)
    axs[1].set_ylim(0,10)

    plt.legend()
    
    axs[0].xaxis.set_minor_locator(AutoMinorLocator(2))
    axs[0].yaxis.set_minor_locator(AutoMinorLocator(2))
    axs[1].xaxis.set_minor_locator(AutoMinorLocator(2))
    axs[1].yaxis.set_minor_locator(AutoMinorLocator(2))

    plt.legend()

    plt.subplots_adjust(wspace=0.1, hspace=0)

    plt.tight_layout()

    plt.savefig(plot_dir+'wigner_lr_sidebyside-%d.pdf'%(idx))
    plt.show()

In [ ]:
if load_data:
    dx_wig_jk_n = full_data['dx_wig_jk_n']
    print(dx_wig_jk_n.shape)
else:
    dx_wig_jk_n = np.empty((len(ns), n_run*n_seed, (dim_x-2)//2))

    lr = 1e-1

    for idx in range(finflag[0], finflag[1], finflag[2]):
        kap = eig_phi[idx]
        for k in range(len(eig_hist_n)):
            print(ns[k])

            y = eig_hist_n[k, :, -1, idx:idx+2]
            s_ens = np.abs(y[:,0] - y[:,1])

            y_jk = np.empty((len(s_ens), len(s_ens)-1))
            for jk in range(len(s_ens)):
                y_jk[jk] = np.r_[s_ens[:jk],s_ens[jk+1:]]

            print(np.mean(y_jk), np.mean(s_ens))

            for jk in range(len(s_ens)):
                n_bins, xran = f_bin(y_jk[jk])

                hb, pb = np.histogram(y_jk[jk], range=xran,
                                      bins=n_bins, density=True)

                px = np.empty_like(hb)
                for i in range(len(px)):
                    px[i] = 0.5*(pb[i] + pb[i+1])

                s_mean = np.sum(px * hb)/np.sum(hb)
                dx_wig_jk_n[k][jk][idx//2] = s_mean

if load_data:
    dx_wig_jk_lr = full_data['dx_wig_jk_lr']
    print(dx_wig_jk_lr.shape)
else:
    dx_wig_jk_lr = np.empty((len(lrs), n_run*n_seed, (dim_x-2)//2))

    batch_size = 16
    for idx in range(finflag[0], finflag[1], finflag[2]):
        kap = eig_phi[idx]

        for k in range(len(eig_hist_lr)):
            print(lrs[k])

            y = eig_hist_lr[k, :, -1, idx:idx+2]
            s_ens = np.abs(y[:,0] - y[:,1])

            y_jk = np.empty((len(s_ens), len(s_ens)-1))
            for jk in range(len(s_ens)):
                y_jk[jk] = np.r_[s_ens[:jk], s_ens[jk+1:]]

            print(np.mean(y_jk), np.mean(s_ens))

            for jk in range(len(s_ens)):
                n_bins, xran = f_bin(y_jk[jk])

                hb, pb = np.histogram(y_jk[jk], range=xran,
                                      bins=n_bins, density=True)

                px = np.empty_like(hb)
                for i in range(len(px)):
                    px[i] = 0.5*(pb[i] + pb[i+1])

                s_mean = np.sum(px * hb)/np.sum(hb)
                dx_wig_jk_lr[k][jk][idx//2] = s_mean

In [ ]:
dx_pred_n = np.empty((len(ns), (N-2)//2))
for i, val in enumerate(ns):
    for idx in range(finflag[0],finflag[1],finflag[2]):
        fac = (mi**2 - eig_phi[idx])
#         fac = 1.
        dx_pred_n[i][idx//2] = np.sqrt(fixed_lr * fac / val)
dx_pred_n = dx_pred_n.flatten()
        
dx_pred_lr = np.empty((len(lrs), (N-2)//2))
for i, val in enumerate(lrs):
    for idx in range(finflag[0],finflag[1],finflag[2]):
        fac = (mi**2 - eig_phi[idx])
#         fac = 1.
        dx_pred_lr[i][idx//2] = np.sqrt(val * fac / fixed_bs)
dx_pred_lr = dx_pred_lr.flatten()

dx_mean_two_n = np.mean(dx_wig_jk_n, axis=1).flatten()
dx_std_two_n = (np.std(dx_wig_jk_n, axis=1)*np.sqrt(dx_wig_jk_n.shape[1] - 1)).flatten()

dx_mean_two_lr = np.mean(dx_wig_jk_lr, axis=1).flatten()
dx_std_two_lr = (np.std(dx_wig_jk_lr, axis=1)*np.sqrt(dx_wig_jk_lr.shape[1] - 1)).flatten()

def lin_fit(x, a):
    return a*x

x = np.r_[dx_pred_n, dx_pred_lr]
y = np.r_[dx_mean_two_n, dx_mean_two_lr]
err = np.r_[dx_std_two_n, dx_std_two_lr]
print(x.shape, y.shape)
popt, pcov = sp.optimize.curve_fit(lin_fit,
                                  xdata=x,
                                  ydata=y,
                                  sigma=err)

print(popt, pcov)
# print(dx_mean_two, dx_std_two)

r = (y - lin_fit(x,*popt))
sigma = np.linalg.inv(np.diag(err))
chisq = r.T @ sigma @ r
dof = len(x) - 2
print(chisq/dof)

x = np.linspace(0, 0.8, 100)
plt.plot(x, lin_fit(x, *popt), 'k--', lw=0.8, )
idx_plot = np.argsort(dx_pred_n)
dx_pred_n = dx_pred_n[idx_plot][:-8]
dx_mean_two_n = dx_mean_two_n[idx_plot][:-8]
dx_std_two_n = dx_std_two_n[idx_plot][:-8]

plt.errorbar(dx_pred_n, dx_mean_two_n, marker='o', ls='',
             yerr=dx_std_two_n, c='navy', label=r'$|\mathcal{B}|$')
plt.errorbar(dx_pred_lr, dx_mean_two_lr, marker='x', ls='',
             yerr=dx_std_two_lr, c='crimson', label=r'$\alpha$')

plt.plot()
plt.xlabel(r'$\sqrt{\frac{\alpha}{|\mathcal{B}|}\kappa^2 \Omega}$')
plt.ylabel(r'$\langle S \rangle$')
    
ytik = np.round(np.arange(0, 1., 0.1),1)
plt.yticks(ytik,ytik)

plt.text(0.95,0.14,
         transform=plt.gca().transAxes,
         horizontalalignment='right', verticalalignment='top',
         s=r"$a_{\rm fit} = %.4f \pm %.4f$"%(popt[0], np.sqrt(pcov[0,0])))

plt.xlim(0,0.2)
plt.ylim(0,0.5)

plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))

plt.legend(loc='upper left')

plt.tight_layout()
plt.savefig(plot_dir+'pred_comp-all_wig-fit-k2_no_off.pdf')
plt.show()

# spectral density

In [ ]:
def rho(x, x0, sig):
    A = np.exp(-(x - x0)**2 / (2.*sig**2))/(4.*np.sqrt(np.pi)*sig)
    B = 2.*np.exp(-(x - x0)**2 / (2.*sig**2))
    C = np.sqrt(2.*np.pi) * (x - x0) * sp.special.erf((x-x0)/(np.sqrt(2)*sig)) / sig
    return A * (B + C)

def Gaussian(x, x0, sig):
    return np.exp(-0.5 * (x - x0)**2 / (sig**2)) / np.sqrt(2 * np.pi * sig**2)

In [ ]:
if load_data:
    sig_list_two_jk_n = full_data['sig_list_two_jk_n']
    x0_list_two_jk_n = full_data['x0_list_two_jk_n']
else:
    x0_list_two_jk_n = np.empty((len(ns), n_run*n_seed*2, num_split))
    sig_list_two_jk_n = np.empty((len(ns), n_run*n_seed*2, num_split))
    sig_err_list_two_jk_n = np.empty((len(ns), n_run*n_seed*2, num_split))
    sigp_list_two_n = np.empty((len(ns)))

    for idx in range(finflag[0], finflag[1], finflag[2]):
        for k in range(len(eig_hist_n)):
            print('n:', ns[k])
            kap = eig_phi[idx]

            print('kap:',kap)
            y = mi**2 - eig_hist_n[k][:,-1,idx:idx+2].flatten()
            print('y:',np.mean(y))


            omg = (mi**2 - kap)/kap
            sig_pred = np.sqrt(fixed_lr * kap * omg**2 / ns[k])
            sigp_list_two_n[k] = sig_pred

            y_jk = np.empty((len(y), len(y)-1))
            for jk in range(len(y)):
                y_jk[jk] = np.r_[y[:jk],y[jk+1:]]

            print(np.mean(y_jk), np.mean(y))

            for jk in range(len(y)):
                n_bins, xran = f_bin(y_jk[jk])

                hb, pb = np.histogram(y_jk[jk], range=xran,
                                      bins=n_bins, density=True)

                px = np.empty_like(hb)
                for i in range(len(px)):
                    px[i] = 0.5*(pb[i] + pb[i+1])

                popt, pcov = curve_fit(rho, px, hb, \
                                       p0=[kap, 1.])

                dx_pred = np.abs(popt[1])

                x0_list_two_jk_n[k][jk][idx//2] = popt[0]
                sig_list_two_jk_n[k][jk][idx//2] = dx_pred
                sig_err_list_two_jk_n[k][jk][idx//2] = pcov[1,1]

            print(np.mean(sig_err_list_two_jk_n[k,:,idx//2], axis=0))
            print(np.mean(sig_list_two_jk_n[k,:,idx//2], axis=0))
            print(sigp_list_two_n[k])

# Non degenerate modes

In [ ]:
if load_data:
    sig_list_one_jk_n = full_data['sig_list_one_jk_n']
    x0_list_one_jk_n = full_data['x0_list_one_jk_n']
else:
    x0_list_one_jk_n = np.empty((len(ns), n_run*n_seed, 2))
    sig_list_one_jk_n = np.empty((len(ns), n_run*n_seed, 2))
    sig_err_list_one_jk_n = np.empty((len(ns), n_run*n_seed, 2))
    sigp_list_one_n = np.empty((len(ns)))

    for idx in [0, -1]:
        for k in range(len(eig_hist_n)):
            print('n:', ns[k])
            kap = eig_phi[idx]

            print('kap:',kap)
            y = mi**2 - eig_hist_n[k][:,-1,idx].flatten()
            print('y:',np.mean(y))

            omg = (mi**2 - kap)/kap
            sig_pred = np.sqrt(fixed_lr * kap * omg**2 / ns[k])
            sigp_list_one_n[k] = sig_pred

            y_jk = np.empty((len(y), len(y)-1))
            for jk in range(len(y)):
                y_jk[jk] = np.r_[y[:jk],y[jk+1:]]

            print(np.mean(y_jk), np.mean(y))

            for jk in range(len(y)):
                n_bins, xran = f_bin(y_jk[jk])

                hb, pb = np.histogram(y_jk[jk], range=xran,
                                      bins=n_bins, density=True)

                px = np.empty_like(hb)
                for i in range(len(px)):
                    px[i] = 0.5*(pb[i] + pb[i+1])

                popt, pcov = curve_fit(Gaussian, px, hb, \
                                       p0=[kap, 1.])#, \

                dx_pred = np.abs(popt[1])

                x0_list_one_jk_n[k][jk][idx] = popt[0]
                sig_list_one_jk_n[k][jk][idx] = dx_pred
                sig_err_list_one_jk_n[k][jk][idx] = pcov[1,1]

            print(np.mean(sig_err_list_one_jk_n[k,:,idx], axis=0))
            print(np.mean(sig_list_one_jk_n[k,:,idx], axis=0))
            print(sigp_list_one_n[k])

In [ ]:
if load_data:
    sig_list_two_jk_lr = full_data['sig_list_two_jk_lr']
    x0_list_two_jk_lr = full_data['x0_list_two_jk_lr']
else:
    x0_list_two_jk_lr = np.empty((len(lrs), n_run*n_seed*2, num_split))
    sig_list_two_jk_lr = np.empty((len(lrs), n_run*n_seed*2, num_split))
    sig_err_list_two_jk_lr = np.empty((len(lrs), n_run*n_seed*2, num_split))
    sigp_list_two_lr = np.empty((len(lrs)))

    for idx in range(finflag[0], finflag[1], finflag[2]):
        for k in range(len(eig_hist_lr)):
            print('n:', lrs[k])
            kap = eig_phi[idx]

            print('kap:',kap)
            y = mi**2 - eig_hist_lr[k][:,-1,idx:idx+2].flatten()
            print('y:',np.mean(y))

            sig_pred = np.sqrt(lrs[k] * kap * omg**2 / fixed_bs)
            sigp_list_two_lr[k] = sig_pred

            y_jk = np.empty((len(y), len(y)-1))
            for jk in range(len(y)):
                y_jk[jk] = np.r_[y[:jk],y[jk+1:]]

            print(np.mean(y_jk), np.mean(y))

            for jk in range(len(y)):
                n_bins, xran = f_bin(y_jk[jk])

                hb, pb = np.histogram(y_jk[jk], range=xran,
                                      bins=n_bins, density=True)

                px = np.empty_like(hb)
                for i in range(len(px)):
                    px[i] = 0.5*(pb[i] + pb[i+1])

                popt, pcov = curve_fit(rho, px, hb, \
                                       p0=[kap, 1.])

                dx_pred = np.abs(popt[1])

                x0_list_two_jk_lr[k][jk][idx//2] = popt[0]
                sig_list_two_jk_lr[k][jk][idx//2] = dx_pred
                sig_err_list_two_jk_lr[k][jk][idx//2] = pcov[1,1]

            print(np.mean(sig_err_list_two_jk_lr[k,:,idx//2], axis=0))
            print(np.mean(sig_list_two_jk_lr[k,:,idx//2], axis=0))
            print(sigp_list_two_lr[k])

# Non-deg modes

In [ ]:
if load_data:
    sig_list_one_jk_lr = full_data['sig_list_one_jk_lr']
    x0_list_one_jk_lr = full_data['x0_list_one_jk_lr']
else:
    x0_list_one_jk_lr = np.empty((len(lrs), n_run*n_seed, 2))
    sig_list_one_jk_lr = np.empty((len(lrs), n_run*n_seed, 2))
    sig_err_list_one_jk_lr = np.empty((len(lrs), n_run*n_seed, 2))
    sigp_list_one_lr = np.empty((len(lrs)))

    for idx in [0, -1]:
        for k in range(len(eig_hist_lr)):
            print('lr:', lrs[k])
            kap = eig_phi[idx]

            print('kap:',kap)
            y = mi**2 - eig_hist_lr[k][:,-1,idx].flatten()
            print('y:',np.mean(y))

            omg = (mi**2 - kap)/kap
            sig_pred = np.sqrt(lrs[k] * kap * omg**2 / fixed_bs)
            sigp_list_one_lr[k] = sig_pred

            y_jk = np.empty((len(y), len(y)-1))
            for jk in range(len(y)):
                y_jk[jk] = np.r_[y[:jk],y[jk+1:]]

            print(np.mean(y_jk), np.mean(y))

            for jk in range(len(y)):
                n_bins, xran = f_bin(y_jk[jk])

                hb, pb = np.histogram(y_jk[jk], range=xran,
                                      bins=n_bins, density=True)
                px = np.empty_like(hb)
                for i in range(len(px)):
                    px[i] = 0.5*(pb[i] + pb[i+1])

                popt, pcov = curve_fit(Gaussian, px, hb, \
                                       p0=[kap, 1.])

                dx_pred = np.abs(popt[1])

                x0_list_one_jk_lr[k][jk][idx] = popt[0]
                sig_list_one_jk_lr[k][jk][idx] = dx_pred
                sig_err_list_one_jk_lr[k][jk][idx] = pcov[1,1]

            print(np.mean(sig_err_list_one_jk_lr[k,:,idx], axis=0))
            print(np.mean(sig_list_one_jk_lr[k,:,idx], axis=0))
            print(sigp_list_one_lr[k])

In [ ]:
sig_pred_n = np.empty((len(ns), (N-2)//2))
for i, val in enumerate(ns):
    for idx in range(finflag[0],finflag[1],finflag[2]):
        Omg = (mi**2 - eig_phi[idx])/eig_phi[idx]**2
        sig_pred_n[i][idx//2] = np.sqrt(fixed_lr * eig_phi[idx]**2 * Omg/val)
sig_pred_n = sig_pred_n.flatten()
        
sig_pred_lr = np.empty((len(lrs), (N-2)//2))
for i, val in enumerate(lrs):
    for idx in range(finflag[0],finflag[1],finflag[2]):
        Omg = (mi**2 - eig_phi[idx])/eig_phi[idx]**2
        sig_pred_lr[i][idx//2] = np.sqrt(val * eig_phi[idx]**2 * Omg/fixed_bs)
sig_pred_lr = sig_pred_lr.flatten()

sig_mean_two_n = np.mean(sig_list_two_jk_n, axis=1).flatten()
sig_std_two_n = (np.std(sig_list_two_jk_n, axis=1)*np.sqrt(sig_list_two_jk_n.shape[1] - 1)).flatten()

sig_mean_two_lr = np.mean(sig_list_two_jk_lr, axis=1).flatten()
sig_std_two_lr = (np.std(sig_list_two_jk_lr, axis=1)*np.sqrt(sig_list_two_jk_lr.shape[1] - 1)).flatten()

def lin_fit(x, a):
    return a*x

x = np.r_[sig_pred_n, sig_pred_lr]
y = np.sqrt(np.pi)*np.r_[sig_mean_two_n, sig_mean_two_lr]
err = np.r_[sig_std_two_n, sig_std_two_lr]
print(x.shape, y.shape)
popt, pcov = sp.optimize.curve_fit(lin_fit,
                                   xdata=x,
                                   ydata=y,
                                   sigma=err)

print(popt, pcov)

r = (y - lin_fit(x,*popt))
sigma = np.linalg.inv(np.diag(err))
chisq = r.T @ sigma @ r
dof = len(x) - 2
print(chisq/dof)

xmax = 0.8
x = np.linspace(0, xmax, 100)
plt.plot(x, lin_fit(x, *popt), 'k--', lw=0.8,)

idx_plot = np.argsort(sig_pred_n)
sig_pred_n = sig_pred_n[idx_plot][:-8]
sig_mean_two_n = sig_mean_two_n[idx_plot][:-8]
sig_std_two_n = sig_std_two_n[idx_plot][:-8]

plt.errorbar(sig_pred_n, np.sqrt(np.pi)*sig_mean_two_n,
             c='navy', marker='o', ls='', 
             yerr=sig_std_two_n.flatten(), 
             label=r'$|\mathcal{B}|$')

plt.errorbar(sig_pred_lr, np.sqrt(np.pi)*sig_mean_two_lr, 
             c='crimson', marker='x', ls='', 
             yerr=sig_std_two_lr.flatten(), 
             label=r'$\alpha$')

plt.xlabel(r'$\sqrt{\frac{\alpha}{|\mathcal{B}|} \kappa^2 \Omega}$')
plt.ylabel(r'$\sqrt{\pi} \sigma$')
    
tik = np.round(np.arange(0,0.6,0.1),1)
plt.yticks(tik,tik)

plt.text(0.95,0.14,
         transform=plt.gca().transAxes,
         horizontalalignment='right', verticalalignment='top',
         s=r"$a_{\rm fit} = %.4f \pm %.4f$"%(popt[0], np.sqrt(pcov[0,0])))

# scalar-fine
plt.xlim(0,0.2)
plt.ylim(0,0.5)

plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))

plt.legend(loc='upper left')

plt.tight_layout()
plt.savefig(plot_dir+'pred_comp-all_sig-fit2-k2_no_off.pdf')
plt.show()

In [ ]:
sqpi = np.sqrt(np.pi)

dx_mean_two_n = (np.mean(dx_wig_jk_n, axis=1)).flatten()
dx_std_two_n = (np.std(dx_wig_jk_n, axis=1)*np.sqrt(dx_wig_jk_n.shape[1] - 1)).flatten()

dx_mean_two_lr = (np.mean(dx_wig_jk_lr, axis=1)).flatten()
dx_std_two_lr = (np.std(dx_wig_jk_lr, axis=1)*np.sqrt(dx_wig_jk_lr.shape[1] - 1)).flatten()

sig_mean_two_n = np.mean(sig_list_two_jk_n, axis=1).flatten()
sig_std_two_n = (np.std(sig_list_two_jk_n, axis=1)*np.sqrt(sig_list_two_jk_n.shape[1] - 1)).flatten()

sig_mean_two_lr = np.mean(sig_list_two_jk_lr, axis=1).flatten()
sig_std_two_lr = (np.std(sig_list_two_jk_lr, axis=1)*np.sqrt(sig_list_two_jk_lr.shape[1] - 1)).flatten()

plt.plot([0.0, 0.8], [0.0, 0.8], 'k--', lw=0.8)

idx_plot = np.argsort(sig_mean_two_n)
sig_mean_two_n = sig_mean_two_n[idx_plot][:-8]
dx_mean_two_n = dx_mean_two_n[idx_plot][:-8]
sig_std_two_n = sig_std_two_n[idx_plot][:-8]
dx_std_two_n = dx_std_two_n[idx_plot][:-8]

plt.errorbar(sqpi*sig_mean_two_n, dx_mean_two_n,
             c='navy', marker='o', ls='', 
             xerr=sig_std_two_n, 
             yerr=dx_std_two_n,
             label=r'$|\mathcal{B}|$')

plt.errorbar(sqpi*sig_mean_two_lr, dx_mean_two_lr,
             c='crimson', marker='x', ls='', 
             xerr=sig_std_two_lr, 
             yerr=dx_std_two_lr,
             label=r'$\alpha$')

plt.plot()
plt.xlabel(r'$\sqrt{\pi}\sigma$')
plt.ylabel(r'$\langle S \rangle$')
    
tik = np.round(np.arange(0,0.6,0.1),1)
plt.xticks(tik,tik)
plt.yticks(tik,tik)

plt.xlim(0,0.5)
plt.ylim(0,0.5)

plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))

plt.legend(loc='upper left')

plt.tight_layout()
plt.savefig(plot_dir+'sig-S.pdf')
plt.show()

S from Wigner semicircle  
sig from spectral density

In [ ]:
x0_mean_lr_deg = np.mean(x0_list_two_jk_lr, axis=1)
x0_std_lr_deg = np.std(x0_list_two_jk_lr, axis=1)*np.sqrt(n_seed*n_run - 1.)
x0_mean_n_deg = np.mean(x0_list_two_jk_n, axis=1)
x0_std_n_deg = np.std(x0_list_two_jk_n, axis=1)*np.sqrt(n_seed*n_run - 1.)

x0_mean_lr_nd = np.mean(x0_list_one_jk_lr, axis=1)
x0_std_lr_nd = np.std(x0_list_one_jk_lr, axis=1)*np.sqrt(n_seed*n_run - 1.)
x0_mean_n_nd = np.mean(x0_list_one_jk_n, axis=1)
x0_std_n_nd = np.std(x0_list_one_jk_n, axis=1)*np.sqrt(n_seed*n_run - 1.)

x0_all_lr = np.c_[x0_mean_lr_nd.T[0], x0_mean_lr_deg, x0_mean_lr_nd.T[-1]]
x0_all_n = np.c_[x0_mean_n_nd.T[0], x0_mean_n_deg, x0_mean_n_nd.T[-1]]

x0_all_lr_err = np.c_[x0_std_lr_nd.T[0], x0_std_lr_deg, x0_std_lr_nd.T[-1]]
x0_all_n_err = np.c_[x0_std_n_nd.T[0], x0_std_n_deg, x0_std_n_nd.T[-1]]

x0_mean_lr0 = x0_all_lr/np.unique(eig_phi).reshape(1,-1)
x0_mean_n0 = x0_all_n/np.unique(eig_phi).reshape(1,-1)

x0_std_lr0 = x0_all_lr_err/np.sqrt(np.unique(eig_phi).reshape(1,-1))
x0_std_n0 = x0_all_n_err/np.sqrt(np.unique(eig_phi).reshape(1,-1))

x_plot_lr = [x/fixed_bs for x in lrs]
x_plot_bs = [fixed_lr/x for x in ns]

for i in range(len(np.unique(eig_phi))):
    plt.errorbar(x_plot_bs[2:], x0_mean_n0.T[i].T[2:],
                 yerr=x0_std_n0.T[i].T[2:],
                 ls=':', marker='o', c='navy', 
                 clip_on=False)

plt.errorbar(x_plot_bs[2:], x0_mean_n0.T[0].T[2:],
             yerr=x0_std_n0.T[0].T[2:],
             ls=':', marker='o', c='navy', 
             label=r"$|\mathcal{B}|$",
             clip_on=False)

for i in range(len(np.unique(eig_phi))):
    plt.errorbar(x_plot_lr, x0_mean_lr0.T[i].T,
                 yerr=x0_std_lr0.T[i].T,
                 ls=':', marker='x', c='crimson',
                 clip_on=False)

plt.errorbar(x_plot_lr, x0_mean_lr0.T[0].T,
             yerr=x0_std_lr0.T[0].T,
             ls=':', marker='x', c='crimson',
             label=r"$\alpha$",
             clip_on=False)

plt.axhline(1, c='k', ls='--')

plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))

plt.xlim(0,0.008)
plt.ylim(0.94,1.04)

plt.ylabel(r"$\lambda_{\rm fit} / \kappa$")
plt.xlabel(r"$\alpha/|\mathcal{B}|$")

plt.legend(loc='lower left')

plt.tight_layout()

plt.savefig(plot_dir+'inter_mode2.pdf')
plt.show()

In [ ]:
if not load_data:
    np.savez(data_dir+'data.npz',
             eig_hist_lr=eig_hist_lr, eig_hist_n=eig_hist_n,
             dx_wig_jk_n=dx_wig_jk_n, dx_wig_jk_lr=dx_wig_jk_lr,
             sig_list_two_jk_n=sig_list_two_jk_n, sig_list_two_jk_lr=sig_list_two_jk_lr,
             x0_list_two_jk_lr=x0_list_two_jk_lr, x0_list_two_jk_n=x0_list_two_jk_n, 
             x0_list_one_jk_lr=x0_list_one_jk_lr, x0_list_one_jk_n=x0_list_one_jk_n,
             sig_list_one_jk_lr=sig_list_one_jk_lr, sig_list_one_jk_n=sig_list_one_jk_n)

    print("Full data saved:", data_dir+'data.npz')